## Python file to clean BQ DB

In [1]:
#Import des librairies nécessaires
import pandas as pd
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [ ]:
# Inspection des données 
QUERY = """
SELECT *
FROM `dash-vlille.data_table_vlille_59000.table_2025`
WHERE is_modified IS NULL
LIMIT 100
"""
query_job = client.query(QUERY)
results = query_job.result()
df = results.to_dataframe()

c:\Users\coulo\VLille\V-Lille-Dash\lab\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   ID_station                    10 non-null     object        
 1   geometry_coordinates          10 non-null     object        
 2   properties_nom                10 non-null     object        
 3   properties_adresse            10 non-null     object        
 4   properties_commune            10 non-null     object        
 5   properties_etat               10 non-null     object        
 6   properties_nb_places_dispo    10 non-null     Int64         
 7   properties_nb_velos_dispo     10 non-null     Int64         
 8   properties_etat_connexion     10 non-null     object        
 9   properties_x                  10 non-null     float64       
 10  properties_y                  10 non-null     float64       
 11  properties_date_modification  10 no

,ID_station,geometry_coordinates,properties_nom,properties_adresse,properties_commune,properties_etat,properties_nb_places_dispo,properties_nb_velos_dispo,properties_etat_connexion,properties_x,properties_y,properties_date_modification,Date_Scrapping,Heure_Min_Scrapping,Date_Paris,is_modified
0,10,"[3.062471, 50.63589]",RIHOUR,28 place Rihour,Lille,EN SERVICE,8,24,CONNECTÉ,3.062471,50.63589,2025-09-02T09:55:10.505+00:00,2025-09-02,07:56:00,2025-09-02 09:55:10.505,<NA>
1,10,"[3.062471, 50.63589]",RIHOUR,28 place Rihour,Lille,EN SERVICE,8,24,CONNECTÉ,3.062471,50.63589,2025-09-02T10:25:11.319+00:00,2025-09-02,08:27:00,2025-09-02 10:25:11.319,<NA>
2,10,"[3.062471, 50.63589]",RIHOUR,28 place Rihour,Lille,EN SERVICE,7,25,CONNECTÉ,3.062471,50.63589,2025-09-02T10:47:08.743+00:00,2025-09-02,08:47:00,2025-09-02 10:47:08.743,<NA>
3,10,"[3.062471, 50.63589]",RIHOUR,28 place Rihour,Lille,EN SERVICE,7,25,CONNECTÉ,3.062471,50.63589,2025-09-02T10:57:09.623+00:00,2025-09-02,08:58:00,2025-09-02 10:57:09.623,<NA>
4,10,"[3.062471, 50.63589]",RIHOUR,28 place Rihour,Lille,EN SERVICE,6,26,CONNECTÉ,3.062471,50.63589,2025-09-02T11:23:12.771+00:00,2025-09-02,09:24:00,2025-09-02 11:23:12.771,<NA>


In [4]:
df_copy = df.copy(deep=True)

In [9]:
# Nettoyage et transformation des données
#Ajout d'une colonne pour marquer les lignes modifiées
#Ajout de la colonne Date_Paris
#Cleaning des dates et retrait de texte inutile
QUERY = """
-- Ajoute une colonne Date_Paris au format DATETIME
# ALTER TABLE `dash-vlille.data_table_vlille_59000.table_2025`
# ADD COLUMN Date_Paris DATETIME,
# ADD COLUMN is_modified BOOL;

-- Met à jour la colonne avec la conversion à l'heure de Paris
UPDATE `dash-vlille.data_table_vlille_59000.table_2025`

SET Date_Paris = DATETIME(TIMESTAMP(properties_date_modification), "Europe/Paris"),
    Date_Scrapping = FORMAT_DATE("%d-%m-%Y", PARSE_DATE("%d/%m/%Y", REPLACE(Date_Scrapping, '_', '/'))),
    ID_station = REPLACE(ID_station, "vlille_temps_reel.", ""),
    Heure_Min_Scrapping = FORMAT_TIME("%H:%M", PARSE_TIME("%H:%M", REPLACE(REPLACE(Heure_Min_Scrapping, "_", ":"),".c", ""))),
    is_modified = TRUE
WHERE is_modified IS NULL;
"""
query_job = client.query(QUERY)
results = query_job.result()
df = results.to_dataframe()


In [10]:
QUERY = """
SELECT *
FROM `dash-vlille.data_table_vlille_59000.table_2025`
WHERE IS_MODIFIED = TRUE
LIMIT 10
"""
query_job = client.query(QUERY)
results = query_job.result()
df_modified = results.to_dataframe()


c:\Users\coulo\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


## Tout récupérer et save en local

In [5]:
#Récupération de toute la base 
df = client.list_rows("dash-vlille.data_table_vlille_59000.table_2025").to_dataframe()

c:\Users\coulo\VLille\V-Lille-Dash\lab\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [7]:
df.to_parquet("data_vlille_2025.parquet", index=False)

In [10]:
df.head()

,ID_station,geometry_coordinates,properties_nom,properties_adresse,properties_commune,properties_etat,properties_nb_places_dispo,properties_nb_velos_dispo,properties_etat_connexion,properties_x,properties_y,properties_date_modification,Date_Scrapping,Heure_Min_Scrapping,Date_Paris,is_modified
0,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T09:25:17.446+00:00,14-03-2025,08:26,2025-03-14 10:25:17.446,True
1,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-11-05T08:45:11.429+00:00,05-11-2025,07:45,2025-11-05 09:45:11.429,True
2,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T09:41:18.075+00:00,14-03-2025,08:42,2025-03-14 10:41:18.075,True
3,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T11:16:16.070+00:00,14-03-2025,10:16,2025-03-14 12:16:16.070,True
4,308,"[2.994654, 50.611307]",SADI CARNOT,178 rue Sadi Carnot,Haubourdin,EN SERVICE,0,20,CONNECTÉ,2.994654,50.611307,2025-03-14T22:15:19.907+00:00,14-03-2025,21:15,2025-03-14 23:15:19.907,True
